In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [2]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
### Load NER model
model_ner = spacy.load('./output/model-best/')

In [5]:
image = cv2.imread('./data/1.jpg')

tessData = pytesseract.image_to_data(image)

tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,653,977,-1,
1,2,1,1,0,0,0,42,49,549,44,-1,
2,3,1,1,1,0,0,42,49,549,44,-1,
3,4,1,1,1,1,0,42,49,549,44,-1,
4,5,1,1,1,1,1,42,49,81,34,95,Cell
...,...,...,...,...,...,...,...,...,...,...,...,...
187,2,1,42,0,0,0,650,292,1,683,-1,
188,3,1,42,1,0,0,650,292,1,683,-1,
189,4,1,42,1,1,0,650,292,1,683,-1,
190,5,1,42,1,1,1,650,292,1,683,95,


In [6]:
# Load Image
image = cv2.imread('./data/1.jpg')

# cv2.imshow('businesscard',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# extract data using Pytesseract 
tessData = pytesseract.image_to_data(image)
# convert into dataframe
tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)

# convet data into content
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)
# get prediction from NER model
doc = model_ner(content)

cell phone repair center mobile repair thomas atkins general manager mobile 91 908725526 mail thomasatkins@gmail.com 91 908725666 website www.abccompany.com ar es ripti aie


In [7]:
image = cv2.imread('./data/1.jpg')

tessData = pytesseract.image_to_data(image)

tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True)
df['text'] = df['text'].apply(cleanText)

df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)

doc = model_ner(content)

cell phone repair center mobile repair thomas atkins general manager mobile 91 908725526 mail thomasatkins@gmail.com 91 908725666 website www.abccompany.com ar es ripti aie


In [8]:
from spacy import displacy

In [9]:
displacy.serve(doc,style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [10]:
displacy.render(doc,style='ent')

### Tagging

In [11]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [12]:
doc.to_json()

{'text': 'cell phone repair center mobile repair thomas atkins general manager mobile 91 908725526 mail thomasatkins@gmail.com 91 908725666 website www.abccompany.com ar es ripti aie',
 'ents': [{'start': 5, 'end': 10, 'label': 'B-ORG'},
  {'start': 11, 'end': 17, 'label': 'I-ORG'},
  {'start': 18, 'end': 24, 'label': 'I-ORG'},
  {'start': 32, 'end': 38, 'label': 'B-ORG'},
  {'start': 39, 'end': 45, 'label': 'B-NAME'},
  {'start': 46, 'end': 52, 'label': 'I-NAME'},
  {'start': 53, 'end': 60, 'label': 'B-DES'},
  {'start': 61, 'end': 68, 'label': 'I-DES'},
  {'start': 76, 'end': 78, 'label': 'B-PHONE'},
  {'start': 79, 'end': 88, 'label': 'I-PHONE'},
  {'start': 94, 'end': 116, 'label': 'B-EMAIL'},
  {'start': 117, 'end': 119, 'label': 'B-PHONE'},
  {'start': 120, 'end': 129, 'label': 'I-PHONE'},
  {'start': 138, 'end': 156, 'label': 'B-WEB'}],
 'tokens': [{'id': 0, 'start': 0, 'end': 4},
  {'id': 1, 'start': 5, 'end': 10},
  {'id': 2, 'start': 11, 'end': 17},
  {'id': 3, 'start': 18, '

In [13]:
doc_text = docjson['text']

In [14]:
docjson['text']

'cell phone repair center mobile repair thomas atkins general manager mobile 91 908725526 mail thomasatkins@gmail.com 91 908725666 website www.abccompany.com ar es ripti aie'

In [15]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,4,cell
1,1,5,10,phone
2,2,11,17,repair
3,3,18,24,center
4,4,25,31,mobile
5,5,32,38,repair
6,6,39,45,thomas
7,7,46,52,atkins
8,8,53,60,general
9,9,61,68,manager


In [16]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [17]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(20)

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,10,phone,B-ORG
2,2,11,17,repair,I-ORG
3,3,18,24,center,I-ORG
4,4,25,31,mobile,O
5,5,32,38,repair,B-ORG
6,6,39,45,thomas,B-NAME
7,7,46,52,atkins,I-NAME
8,8,53,60,general,B-DES
9,9,61,68,manager,I-DES


In [18]:
# join lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

In [19]:
# inner join with start 
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

In [20]:
dataframe_info.tail(20)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
3,5,1,1,1,1,4,447,50,144,33,96,center,24,18,center,I-ORG
4,5,1,2,1,1,1,39,129,46,11,96,mobile,31,25,mobile,O
5,5,1,2,1,1,2,91,129,46,14,96,repair,38,32,repair,B-ORG
6,5,1,3,1,1,1,6,179,45,10,96,thomas,45,39,thomas,B-NAME
7,5,1,3,1,1,2,56,179,36,10,81,atkins,52,46,atkins,I-NAME
8,5,1,3,1,2,1,7,195,43,10,96,general,60,53,general,B-DES
9,5,1,3,1,2,2,56,195,49,13,95,manager,68,61,manager,I-DES
10,5,1,5,1,1,1,116,218,47,26,95,mobile,75,69,mobile,O
11,5,1,5,1,1,2,201,227,19,10,78,91,78,76,91,B-PHONE
12,5,1,5,1,1,3,227,227,61,10,85,908725526,88,79,908725526,I-PHONE


### Bounding Box

In [22]:
bb_df = dataframe_info.query("label != 'O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
bb_df = dataframe_info.query("label != 'O' ")
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,1,1,1,2,136,49,135,34,95,phone,10,5,phone,B-ORG
2,5,1,1,1,1,3,285,49,148,44,96,repair,17,11,repair,I-ORG
3,5,1,1,1,1,4,447,50,144,33,96,center,24,18,center,I-ORG
5,5,1,2,1,1,2,91,129,46,14,96,repair,38,32,repair,B-ORG
6,5,1,3,1,1,1,6,179,45,10,96,thomas,45,39,thomas,B-NAME
7,5,1,3,1,1,2,56,179,36,10,81,atkins,52,46,atkins,I-NAME
8,5,1,3,1,2,1,7,195,43,10,96,general,60,53,general,B-DES
9,5,1,3,1,2,2,56,195,49,13,95,manager,68,61,manager,I-DES
11,5,1,5,1,1,2,201,227,19,10,78,91,78,76,91,B-PHONE
12,5,1,5,1,1,3,227,227,61,10,85,908725526,88,79,908725526,I-PHONE


In [24]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,1,1,1,2,136,49,135,34,95,phone,10,5,phone,ORG
2,5,1,1,1,1,3,285,49,148,44,96,repair,17,11,repair,ORG
3,5,1,1,1,1,4,447,50,144,33,96,center,24,18,center,ORG
5,5,1,2,1,1,2,91,129,46,14,96,repair,38,32,repair,ORG
6,5,1,3,1,1,1,6,179,45,10,96,thomas,45,39,thomas,NAME


In [25]:
# group the label
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [26]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group
1,5,1,1,1,1,2,136,49,135,34,95,phone,10,5,phone,ORG,1
2,5,1,1,1,1,3,285,49,148,44,96,repair,17,11,repair,ORG,1
3,5,1,1,1,1,4,447,50,144,33,96,center,24,18,center,ORG,1
5,5,1,2,1,1,2,91,129,46,14,96,repair,38,32,repair,ORG,1
6,5,1,3,1,1,1,6,179,45,10,96,thomas,45,39,thomas,NAME,2
7,5,1,3,1,1,2,56,179,36,10,81,atkins,52,46,atkins,NAME,2
8,5,1,3,1,2,1,7,195,43,10,96,general,60,53,general,DES,3
9,5,1,3,1,2,2,56,195,49,13,95,manager,68,61,manager,DES,3
11,5,1,5,1,1,2,201,227,19,10,78,91,78,76,91,PHONE,4
12,5,1,5,1,1,3,227,227,61,10,85,908725526,88,79,908725526,PHONE,4


In [27]:
# right and bottom of bounding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

In [28]:
# tagging: groupby group
col_group = ['left','top','right','bottom','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [29]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

In [30]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,91,591,49,143,[ORG],phone repair center repair
2,6,92,179,189,[NAME],thomas atkins
3,7,105,195,208,[DES],general manager
4,201,288,227,237,[PHONE],91 908725526
5,490,640,228,241,[EMAIL],thomasatkins@gmail.com
6,201,288,243,253,[PHONE],91 908725666
7,490,623,244,257,[WEB],www.abccompany.com


In [31]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    
    label = str(label)
    cv2.putText(img_bb,label,(l,t), cv2.FONT_HERSHEY_PLAIN, 1, (255,0,255),2)
    
    
cv2.imshow('Bounding Box Documents',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Parser

In [38]:
def parser(text,label):
    if label == 'PHONE':
        text = text.lower()
        text = re.sub(r'\D','',text)
        
    elif label == 'EMAIL':
        text = text.lower()
        allow_special_char = '@_.\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
        
    elif label == 'WEB':
        text = text.lower()
        allow_special_char = ':/.%#\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
        
    elif label in ('NAME', 'DES'):
        text = text.lower()
        text = re.sub(r'[^a-z ]','',text)
        text = text.title()
        
    elif label == 'ORG':
        text = text.lower()
        text = re.sub(r'[^a-z0-9 ]','',text)
        text = text.title()
        
    return text

#### Entities

In [39]:
info_array = dataframe_info[['token','label']].values
entities = dict(NAME=[],ORG=[],DES=[],PHONE=[],EMAIL=[],WEB=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]
    
    # step -1 parse the token
    text = parser(token,label_tag)
    
    if bio_tag in ('B','I'):
        
        if previous != label_tag:
            entities[label_tag].append(text)
            
        else:
            if bio_tag == "B":
                entities[label_tag].append(text)
                
            else:
                if label_tag in ("NAME",'ORG','DES'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                    
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
                    
    
    
    previous = label_tag


In [40]:
entities

{'NAME': ['Thomas Atkins'],
 'ORG': ['Phone Repair Center', 'Repair'],
 'DES': ['General Manager'],
 'PHONE': ['91908725526', '91908725666'],
 'EMAIL': ['thomasatkins@gmail.com'],
 'WEB': ['www.abccompany.com']}

In [41]:
parser('Marko 657-764-321','NAME')

'Marko '

In [42]:
parser('Thomas 14thomas51515','NAME')

'Thomas Thomas'

In [43]:
parser('thomas 456634653','PHONE')

'456634653'

In [44]:
parser('George 2626235','NAME')

'George '

In [45]:
parser('John 2626235','NAME')

'John '

In [48]:
parser('2523532 www.abc.com','EMAIL')

'2523532 www.abc.com'